# This is the test notebook
you guys can see what stupid things I have tried here lol

In [19]:
def find_ancestors(tree, target_index):
    ancestors = []
    for i, node in enumerate(tree):
        if i == target_index:
            ancestors.append(i)
            break

        if '(' in node:
            print('add', i, node)
            ancestors.append(i)
        
        for _ in range(0, node.count(')')):
            print('pop', i, node)
            ancestors.pop()

    return ancestors

# Example usage:
tree = ['(S*', '(NP', '*', '*)', '(VP', '*', '(NP', '*', '*))', '(SS', '*)']
target_index = 10

ancestors = find_ancestors(tree, target_index)
print(f"Ancestors of Index {target_index}: {ancestors}")


add 0 (S*
add 1 (NP
pop 3 *)
add 4 (VP
add 6 (NP
pop 8 *))
pop 8 *))
add 9 (SS
Ancestors of Index 10: [0, 9, 10]


In [3]:
import pandas as pd
import numpy as np
from convert_conll import *
from features import *

In [4]:
df = read_conll("converted_train.tsv")
dft = df.head(190) # first 8 sentences

In [5]:
def to_sentences_and_cue_position(df):
    """
    SUPPORT FUNCTION FOR THE FOLLOWING token_cue_distance(df)
    
    Creates two lists: a list of sentences; their corresponding negation cues.
    Both lists consits of the token and its position.
    If the sentence has no negation, cues will be ***
    """
    sent_list, sent, neg_list, neg = [],[],[],[]

    for x in range(len(df)): # x: sentences
        if df['token_id'][x] == '0': # if starting of sentence, clear containers
            sent_list.append(sent)
            sent = []

            cue='' # deal with multi word negation such as "by no means"
            for y in neg: # y: collected column 'negation_word' in sentence x
                if y[0] == '***':
                    neg_list.append(['***','***'])
                    break
                    
                if y[0] != '_':
                    cue = cue + y[0]
                    position = y[1] # define the last word as position (i.e. "means" in "by no means")
            if cue != '':
                neg_list.append([cue,position])
            neg = []
    
        sent.append([df['token'][x] , df['token_id'][x]])
        neg.append([df['negation_word'][x] , df['token_id'][x]]) # collect column 'negation_word' for one sentence

    sent_list.pop(0)
    sent_list.append(sent)
    cue=''
    for y in neg:
        if y[0] == '***':
            neg_list.append(['***','***'])
            break
                    
        if y[0] != '_':
            cue = cue + y[0]
            position = y[1] # define the last word as position (i.e. "means" in "by no means")
    if cue != '':
        neg_list.append([cue,position])

    return sent_list, neg_list

In [6]:
sent_list, neg_list = to_sentences_and_cue_position(df)

In [7]:
def token_cue_distance(df):
    """
    Calculates the distance from every token to the negation cue in this sentence.
    The cue to its own distance is 0.
    If the distance is positive, then the token is befor the cue and vise versa.
    If there's no negation in the sentence, the distance will be ***
    """
    sent_list, neg_list = to_sentences_and_cue_position(df)
    dist_list = []
    for x in range(len(neg_list)): # x: current sentence number
        #print(x)
        for y in range(len(sent_list[x])): # y: current word in the sentence
            if neg_list[x][0] == '***':
                dist_list.append('***')
            else:
                dist = int(sent_list[x][y][1]) - int(neg_list[x][1])
                dist_list.append(dist)
                
    df = df.copy()
    df.loc[:, "cue_distance"] = dist_list
    return df

In [8]:
token_cue_distance(df)

,document_id,sentence_id,token_id,token,lemma,pos_tag,parsing_tree,negation_word,negation_scope,negation_event,cue_distance
0,baskervilles01,0,0,Chapter,Chapter,NN,(NP*,***,_,_,***
1,baskervilles01,0,1,1.,1.,CD,*,***,_,_,***
2,baskervilles01,0,2,Mr.,Mr.,NNP,*,***,_,_,***
3,baskervilles01,0,3,Sherlock,Sherlock,NNP,*,***,_,_,***
4,baskervilles01,0,4,Holmes,Holmes,NNP,*),***,_,_,***
...,...,...,...,...,...,...,...,...,...,...,...
69213,baskervilles14,270,58,slopes,slope,NNS,*),_,_,_,42
69214,baskervilles14,270,59,of,of,IN,(PP*,_,_,_,43
69215,baskervilles14,270,60,the,the,DT,(NP*,_,_,_,44
69216,baskervilles14,270,61,moor,moor,NN,*)))))))))))))))))))))))))))),_,_,_,45


In [9]:
df[~df['token'].str.isalpha()]['token'].unique()

array(['1.', 'Mr.', ',', '.', 'hearth-rug', 'bulbous-headed', '``', "''",
       'M.R.C.S.', 'C.C.H.', '1884', 'old-fashioned', '--', '?',
       'well-polished', 'silver-plated', 'coffee-pot', "'s", 'Dr.',
       'well-esteemed', '!', 'thick-iron', '`', 'C.C.H', "'",
       'self-importance', 'C.C.', 'well-established', 'house-surgeon',
       'house-physician', 'middle-aged', 'absent-minded', '1882',
       'House-surgeon', '(', ')', '1883', 'visiting-card', 'curly-haired',
       'door-step', "n't", 'gold-rimmed', 'frock-coat', 'M.R.C.S',
       'well-marked', 'supra-orbital', ';', '2.', '1730', '1742',
       'breast-pocket', 'strong-minded', ':', 'twenty-four',
       'finger-tips', 'old-world', 'by-word', 'dining-hall', 'dare-devil',
       '[', ']', '14th', 'Mid-Devon', "o'clock", 'moor-gate',
       'horse-dealer', 'post-mortem', 'long-standing', '3.', 'sheep-dogs',
       'sheep-dog', 'wicket-gate', 'summer-house', 'cross-examined',
       'hard-headed', 'hell-hound', '1876', 

In [23]:
def special_token_inbetween(df):
    '''
    Checks if there exist special token between the token and the negation cue.
    Special tokens are: punctuations and some of the conjunctions.
    If the token is the cue, the value is set as False.
    '''
    existence_list = []
    conj_punt_set = [',','.','?','"','\'','!','``',':',';','\'\'','-','--','`','(',')','[',']',
                    'for','and','nor','but','or','yet','while','when','whereas','whenever','wherever','whether','if','because','before',
                     'until','unless','since','so','although','after','as','','Accordingly','After','Also','Besides','Consequently',
                     'Conversely','Finally','Furthermore','Hence','However','Indeed','Instead','Likewise','Meanwhile','Moreover','Nevertheless',
                     'Nonetheless','Otherwise','Similarly','Still','Subsequently','Then','Therefore','Thus','except','rather']
    sent_list, neg_list = to_sentences_and_cue_position(df)
    
    for x in range(len(neg_list)): # x: the sentences                
        if neg_list[x][0] == '***':
            for y in range(len(sent_list[x])): # y: current word in the sentence
                existence_list.append('***')

        else:
            cue_position = int(neg_list[x][1])
            for y in range(0,cue_position+1): # y: tokens before cue and cue
                y_exist = False
                for z in range(y+1,cue_position+1): # z: all tokens between y and cue
                    if sent_list[x][y][0].casefold() in (token.casefold() for token in conj_punt_set):
                        y_exist = True
                if y == cue_position: # token is cue
                    y_exist = False # Set as False in this case
                existence_list.append(y_exist)

            for y in range(cue_position+1,len(sent_list[x])): # y: tokens after cue
                y_exist = False
                for z in range(cue_position+1,len(sent_list[x])): # z: all tokens between cue and y
                    if sent_list[x][y][0].casefold() in (token.casefold() for token in conj_punt_set):
                        y_exist = True
                existence_list.append(y_exist)

    df = df.copy()
    df.loc[:, "cp_existence"] = existence_list
    
    return existence_list

In [1]:
from get_all_features import main as testmain

2024-01-22 16:56:10.144589: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-22 16:56:10.170754: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-22 16:56:10.170776: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-22 16:56:10.171476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-22 16:56:10.175920: I tensorflow/core/platform/cpu_feature_guar

In [2]:
features = testmain('converted_train.tsv')

16it [00:00, 9516.29it/s]
7it [00:00, 9338.46it/s]
6it [00:00, 5065.58it/s]
10it [00:00, 13025.79it/s]
5it [00:00, 7039.79it/s]
15it [00:00, 14706.54it/s]
8it [00:00, 3199.62it/s]
10it [00:00, 4706.36it/s]
25it [00:00, 7455.75it/s]
11it [00:00, 6246.59it/s]
17it [00:00, 11679.47it/s]
17it [00:00, 10892.63it/s]
116it [00:00, 7066.96it/s]
18it [00:00, 11074.88it/s]
17it [00:00, 10510.49it/s]
17it [00:00, 7262.49it/s]
12it [00:00, 7475.37it/s]
38it [00:00, 11979.22it/s]
3it [00:00, 4298.91it/s]
17it [00:00, 9364.74it/s]
39it [00:00, 13991.78it/s]
11it [00:00, 6873.86it/s]
10it [00:00, 8531.94it/s]
11it [00:00, 12741.60it/s]
25it [00:00, 12062.30it/s]
29it [00:00, 5468.70it/s]
25it [00:00, 13403.76it/s]
44it [00:00, 17950.53it/s]
23it [00:00, 10781.07it/s]
14it [00:00, 13502.01it/s]
8it [00:00, 4323.47it/s]
6it [00:00, 11264.92it/s]
11it [00:00, 10220.94it/s]
33it [00:00, 14484.31it/s]
8it [00:00, 7620.81it/s]
9it [00:00, 5006.46it/s]
9it [00:00, 6158.03it/s]
21it [00:00, 10651.88it/s]
7it

FileNotFoundError: [Errno 2] No such file or directory: '/home/arimo/.local/lib/python3.10/site-packages/en_core_web_sm/en_core_web_sm-3.6.0/tokenizer'

In [3]:
import pandas as pd

In [4]:
def read_conll(filename):
    """
    Read data from a CoNLL-formatted file and return it as a DataFrame.

    Args:
        filename (str): The path to the CoNLL-formatted file.

    Returns:
        pd.DataFrame: A DataFrame containing the parsed data with specified column names.

    """
    # Define column names for the DataFrame
    column_names = ['document_id', 'sentence_id', 'token_id', 'token', 'lemma', 'pos_tag', 'parsing_tree', 'negation_word', 'negation_scope', 'negation_event']

    # Read lines from the file
    with open(filename, 'r') as file:
        lines = file.readlines()

    # Initialize a list to store parsed data
    data = []

    # Iterate through lines and parse non-empty lines
    for line in lines:
        line = line.strip()
        if line:
            # Split the line into columns
            columns = line.split('\t')
            data.append(columns)

    # Create a DataFrame using the parsed data and column names
    df = pd.DataFrame(data, columns=column_names)
    return df

In [6]:
inputfile = '../data/starsem-st-2012-data/cd-sco/corpus/training/SEM-2012-SharedTask-CD-SCO-training-09032012.txt'

In [12]:
df_names = list(range(19))
df = pd.read_csv(inputfile, sep='\t',header=None,names=df_names).fillna('_')
df2 = df.loc[~df[10].str.contains("_")]
df3 = df.loc[~df[13].str.contains("_")]
df4 = df.loc[~df[16].str.contains("_")]

In [21]:
print(len(set(zip(df[0], df[1]))), 
      len(set(zip(df2[0], df2[1]))),'{:04.2f}'.format(100*len(set(zip(df2[0], df2[1])))/len(set(zip(df[0], df[1])))),
      len(set(zip(df3[0], df3[1]))),'{:04.2f}'.format(100*len(set(zip(df3[0], df3[1])))/len(set(zip(df[0], df[1])))),
      len(set(zip(df4[0], df4[1]))),'{:04.2f}'.format(100*len(set(zip(df4[0], df4[1])))/len(set(zip(df[0], df[1])))) )

3644 114 3.13 18 0.49 4 0.11


In [23]:
inputfile = '../data/starsem-st-2012-data/cd-sco/corpus/test-gold/SEM-2012-SharedTask-CD-SCO-test-cardboard-GOLD.txt'
inputfile2 = '../data/starsem-st-2012-data/cd-sco/corpus/test-gold/SEM-2012-SharedTask-CD-SCO-test-circle-GOLD.txt'
df_names = list(range(16))
df = pd.read_csv(inputfile, sep='\t',header=None,names=df_names).fillna('_')
df2 = pd.read_csv(inputfile2, sep='\t',header=None,names=df_names).fillna('_')

In [24]:
df=pd.concat([df,df2])

In [26]:
df2 = df.loc[~df[10].str.contains("_")]
df3 = df.loc[~df[13].str.contains("_")]
print(len(set(zip(df[0], df[1]))), 
      len(set(zip(df2[0], df2[1]))),'{:04.2f}'.format(100*len(set(zip(df2[0], df2[1])))/len(set(zip(df[0], df[1])))),
      len(set(zip(df3[0], df3[1]))),'{:04.2f}'.format(100*len(set(zip(df3[0], df3[1])))/len(set(zip(df[0], df[1])))),
      len(set(zip(df4[0], df4[1]))),'{:04.2f}'.format(100*len(set(zip(df4[0], df4[1])))/len(set(zip(df[0], df[1])))) )

1089 28 2.57 1 0.09 4 0.37


In [27]:
inputfile = '../data/starsem-st-2012-data/cd-sco/corpus/dev/SEM-2012-SharedTask-CD-SCO-dev-09032012.txt'
df = pd.read_csv(inputfile, sep='\t',header=None,names=df_names).fillna('_')
df2 = df.loc[~df[10].str.contains("_")]
df3 = df.loc[~df[13].str.contains("_")]
print(len(set(zip(df[0], df[1]))), 
      len(set(zip(df2[0], df2[1]))),'{:04.2f}'.format(100*len(set(zip(df2[0], df2[1])))/len(set(zip(df[0], df[1])))),
      len(set(zip(df3[0], df3[1]))),'{:04.2f}'.format(100*len(set(zip(df3[0], df3[1])))/len(set(zip(df[0], df[1])))),
      len(set(zip(df4[0], df4[1]))),'{:04.2f}'.format(100*len(set(zip(df4[0], df4[1])))/len(set(zip(df[0], df[1])))) )

787 27 3.43 2 0.25 4 0.51
